<a href="https://colab.research.google.com/github/tncourtz/RL-EV-CS-Energy-Management-System/blob/main/PSOV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
uploaded = files.upload()

Saving trading_data.xlsx to trading_data.xlsx


In [4]:
from pandas import read_excel
df = read_excel('trading_data.xlsx')

PVt= df['pv'].values
PLt = df['load'].values
GTt = df['gt'].values

In [ ]:
import random
import numpy as np 
from math import exp

W = 0.5
c1 = 2
c2 = 2

n_iterations = int(input("Inform the number of iterations: "))     #number of iterations
n_particles = int(input("Inform the number of particles: "))       #number of iterations

class Particle():
    def __init__(self, n_particles):
        self.position = np.random.uniform(low=0.1, high=0.95, size=1)       #initialize a soc particle between constraints
        self.pbest_position = self.position.copy()                          #initialize best particle value
        self.pbest_value = float('inf')                                     # intialize the best value to a large number
        self.velocity = np.array(0)                                         #initialize the velocity of the particle

    #print postion
    def __str__(self):
        print("Current Position", self.position, " Best Position ", self.pbest_position)
    #use velocity to move the particle
    def move(self):
        self.position = self.position + self.velocity

###############################################################################################################################
class Space():
    def __init__(self, n_particles):
        self.n_particles = n_particles
        self.particles = []
        self.gbest_value = float('inf')                                         #best global value
        self.gbest_position = np.random.uniform(low=0.1, high=0.95, size=1)     #best global position

    def print_particles(self):
        for particle in self.particles:
            particle.__str__()
    
    # cost function
    def fitness(self, particle, GT, PV, PL): #GT,PV, PL are from arrays of the 24h grid prices, pv, and load
        #Variables 
        Q = 24                               # (kwh) battery full capacity
        Tamb = 23                            #ambient temperature
        Rth = 0.002                          # battery thermal resistance
        # model parameters
        a = 0.5965
        b = 2.040
        m = 3.185
        d = 0.0768
        Yhr = 8760                           # hours per year
        Cbat = 400                           # initial battery cost 

        SOCmin = 0.2
        SOCmax = 0.9
        SOCinit = 0.5
        Pbmin = -18
        Pbmax = 18
        Pgmin = -24
        Pgmax = 24

        #SYSTEM DYNAMICS
        dt =1
        Pb = particle.position*Q                          # battery power
        Pg = PL - PV - Pb                                 #grid suppled power
        T = Tamb + Rth*abs(Pb)                            # battery temperature 
        L = a*Q*exp(b*Q/T)                                # battery lifetime 
        SOCavg = particle.position                        # average SOC ?what is the avg. soc
        DOD = particle.position                           # battery depth of discharge

        #COST FUNCTIONS
        Cg = GT*Pg*dt                                     # grid running cost
        Ctemp = Cbat*dt/(Yhr*L)                           # Temperature operation cost
        Csoc = (Cbat*m*SOCavg-d)/(15*0.2*Yhr)             # Cost due to avgSOC 
        Cdod = (-0.012*DOD**4) + (0.033*DOD**3) + (0.021*DOD**2) - (0.056*DOD) # cost due to dod
        Cbd = max(Ctemp, Cdod, Csoc)                      # total battery degradation cost 

        return Cg + Pb*dt*Cbd                             # total system cost

    # find the best position for the particles
    def set_pbest(self, GT, PV, PL):
        for particle in self.particles:
            i = self.particles.index(particle)              # indexing 
            fitness_cand = self.fitness(particle, GT[i], PV[i], PL[i])
            if(particle.pbest_value > fitness_cand):
                particle.pbest_value = fitness_cand
                particle.pbest_position = particle.position
            
    # find best global position for particles
    def set_gbest(self, GT, PV, PL):
        for particle in self.particles:
            i = self.particles.index(particle)
            best_fitness_cand = self.fitness(particle, GT[i], PV[i], PL[i])
            if(self.gbest_value > best_fitness_cand):
                self.gbest_value = best_fitness_cand
                self.gbest_position = particle.position
    
    #move particles
    def move_particles(self):
        for particle in self.particles:
            global W
            new_velocity = (W*particle.velocity) + (c1*random.random()) * (particle.pbest_position - particle.position) + \
                            (random.random()*c2) * (self.gbest_position - particle.position)
            particle.velocity = new_velocity
            particle.move()

#############################################################################################################################
#IMPLEMENTATION OF PSO ALGO.

search_space = Space(n_particles)
particles_vector = [Particle(n_particles) for i in range(search_space.n_particles)]    
search_space.particles = particles_vector
search_space.print_particles()

iteration = 0
while(iteration < n_iterations):
    search_space.set_pbest(GTt, PVt, PLt)    
    search_space.set_gbest(GTt, PVt, PLt)
    search_space.move_particles()
    iteration += 1
    
print("The best solution is: ", search_space.gbest_position, " in n_iterations: ", iteration)


# **System Dynamics**

In [ ]:
Q = 24  # (kwh) battery full capacity
Tamb = 23 #ambient temperature
Rth = 0.002 # battery thermal resistance
# model parameters
a = 0.5965
b = 2.040
m = 3.185
d = 0.0768
Yhr = 8760 # hours per year

SOCmin = 0.2
SOCmax = 0.9
SOCinit = 0.5
Pbmin = -18
Pbmax = 18
Pgmin = -24
Pgmax = 24


# **System Costs** 

In [ ]:
Cbat = # battery inital cost
DOD = # dod 
GT = # grid tariff
dt = # delta-time for PSO





In [44]:
from __future__ import division
import random
import math
import numpy as np

n_iterations = int(input("Inform the number of iterations: "))     #number of iterations
num_dimensions = int(input("Inform the number of particles: "))       #number of iterations

#--- COST FUNCTION 
# function we are attempting to optimize (minimize)
def func1(particles, GT, PV, PL):
  #Variables 
  Q = 100                               # (kwh) battery full capacity
  Tamb = 23                            #ambient temperature
  Rth = 0.002                          # battery thermal resistance
  # model parameters
  a = 0.5965
  b = 2.040
  m = 3.185
  d = 0.0768
  Yhr = 8760                           # hours per year
  Cbat = 400                           # initial battery cost 

  SOCmin = 0.2
  SOCmax = 0.9
  SOCinit = 0.5
  Pbmin = -18
  Pbmax = 18
  Pgmin = -24
  Pgmax = 24
  dt =1

  total=0
  for particle in particles:
    #SYSTEM DYNAMICS
    Pb = particle*Q                          # battery power
    Pg = PL - PV - Pb                              #grid suppled power
    T = Tamb + Rth*abs(Pb)                            # battery temperature 
    L = a*Q*exp(b*Q/T)                                # battery lifetime 
    SOCavg = particle                        # average SOC ?what is the avg. soc
    DOD = 1 - SOCavg                          # battery depth of discharge
    #COST FUNCTIONS
    Cg = GT*Pg*dt                                     # grid running cost
    Ctemp = Cbat*dt/(Yhr*L)                           # Temperature operation cost
    Csoc = (Cbat*m*SOCavg-d)/(15*0.2*Yhr)             # Cost due to avgSOC 
    Cdod = (-0.012*DOD**4) + (0.033*DOD**3) + (0.021*DOD**2) - (0.056*DOD) # cost due to dod
    Cbd = max(Ctemp, Cdod, Csoc)                      # total battery degradation cost 
    total+= Cg + Pb*dt*Cbd
  return total

#--- MAIN 
class Particle:
    def __init__(self,num_dimensions):
        self.position_i = np.random.uniform(low=0.1, high=0.95, size=num_dimensions)          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))

    # evaluate current fitness
    def evaluate(self,costFunc, GT, PV, PL):
        self.err_i=costFunc(self.position_i,GT, PV, PL)
        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.5       # constant inertia weight (how much to weigh the previous velocity)
        c1=1        # cognative constant
        c2=2        # social constant

        for i in range(0,num_dimensions):
            r1=random.random()
            r2=random.random()

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]

            # adjust maximum position if necessary
            if self.position_i[i]>bounds[1]:
                self.position_i[i]=bounds[1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[0]:
                self.position_i[i]=bounds[0]
                
class PSO():
    def __init__(self,costFunc,x0,bounds,num_particles,maxiter):
        global num_dimensions

        # num_dimensions=len(x0)
        err_best_g=-1                   # best error for group
        pos_best_g=[]                   # best position for group

        # establish the swarm
        swarm=[]
        for i in range(0,num_particles):
            swarm.append(Particle(num_dimensions))


        # begin optimization loop
        i=0
        while i < maxiter:
            #print i,err_best_g
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
                swarm[j].evaluate(costFunc, GTt[j], PVt[j], PLt[j])
                
                # determine if current particle is the best (globally)
                if swarm[j].err_i < err_best_g or err_best_g == -1:
                    pos_best_g=list(swarm[j].position_i)
                    err_best_g=float(swarm[j].err_i)

            # cycle through swarm and update velocities and position
            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position(bounds)
            i+=1

        # print final results
        print ('FINAL:')
        print (pos_best_g)
        print (err_best_g)

if __name__ == "__PSO__":
    main()

#--- EXECUTE

initial=np.random.uniform(low=5, high=10, size=num_dimensions)               # initial starting location [x1,x2...]
bounds=[0.1,0.95]  # input bounds
PSO(func1,initial,bounds,num_particles=15,maxiter=5)


Inform the number of iterations: 100
Inform the number of particles: 24
FINAL:
[0.849831602717156, 0.6846564132254724, 0.9278899145496038, 0.95, 0.2713822336452687, 0.8786761539212613, 0.1, 0.6455950820169192, 0.5328352038739893, 0.44805275300847613, 0.7866305732736911, 0.95, 0.9238582811614526, 0.1, 0.1, 0.5936750614931434, 0.1, 0.84755340076417, 0.1712178361510691, 0.725583150645357, 0.7905341550025443, 0.95, 0.10261278958475836, 0.3002854413612296]
-884.0089886445191


In [41]:
from __future__ import division
import random
import math

#--- COST FUNCTION 
# function we are attempting to optimize (minimize)
def func1(x):
    total=0
    for i in range(len(x)):
        total+=x[i]**2
        print('total')
        print(total)
    return total

#--- MAIN 
class Particle:
    def __init__(self,x0):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness
    def evaluate(self,costFunc):
        self.err_i=costFunc(self.position_i)
        print(self.err_i)
        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.5       # constant inertia weight (how much to weigh the previous velocity)
        c1=1        # cognative constant
        c2=2        # social constant

        for i in range(0,num_dimensions):
            r1=random.random()
            r2=random.random()

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]

            # adjust maximum position if necessary
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i]=bounds[i][0]
                
class PSO():
    def __init__(self,costFunc,x0,bounds,num_particles,maxiter):
        global num_dimensions

        num_dimensions=len(x0)
        err_best_g=-1                   # best error for group
        pos_best_g=[]                   # best position for group

        # establish the swarm
        swarm=[]
        for i in range(0,num_particles):
            swarm.append(Particle(x0))
            print(swarm)

        # begin optimization loop
        i=0
        while i < maxiter:
            #print i,err_best_g
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
                swarm[j].evaluate(costFunc)

                # determine if current particle is the best (globally)
                if swarm[j].err_i < err_best_g or err_best_g == -1:
                    pos_best_g=list(swarm[j].position_i)
                    err_best_g=float(swarm[j].err_i)

            # cycle through swarm and update velocities and position
            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position(bounds)
            i+=1

        # print final results
        print('FINAL:')
        print(pos_best_g)
        print(err_best_g)

if __name__ == "__PSO__":
    main()

#--- EXECUTE

initial=[5,5]               # initial starting location [x1,x2...]
bounds=[(-10,10),(-10,10)]  # input bounds [(x1_min,x1_max),(x2_min,x2_max)...]
PSO(func1,initial,bounds,num_particles=15,maxiter=30)

[<__main__.Particle object at 0x7f3d28240c18>, <__main__.Particle object at 0x7f3d28240be0>]
[<__main__.Particle object at 0x7f3d28240c18>, <__main__.Particle object at 0x7f3d28240be0>, <__main__.Particle object at 0x7f3d28240cc0>]
[<__main__.Particle object at 0x7f3d28240c18>, <__main__.Particle object at 0x7f3d28240be0>, <__main__.Particle object at 0x7f3d28240cc0>, <__main__.Particle object at 0x7f3d28240cf8>]
[<__main__.Particle object at 0x7f3d28240c18>, <__main__.Particle object at 0x7f3d28240be0>, <__main__.Particle object at 0x7f3d28240cc0>, <__main__.Particle object at 0x7f3d28240cf8>, <__main__.Particle object at 0x7f3d28240d30>]
[<__main__.Particle object at 0x7f3d28240c18>, <__main__.Particle object at 0x7f3d28240be0>, <__main__.Particle object at 0x7f3d28240cc0>, <__main__.Particle object at 0x7f3d28240cf8>, <__main__.Particle object at 0x7f3d28240d30>, <__main__.Particle object at 0x7f3d28240d68>]
[<__main__.Particle object at 0x7f3d28240c18>, <__main__.Particle object at

In [2]:
import random
import numpy as np

velocity_i = np.array()

print(len(velocity_i))

2
